# Simulations for the Acetate Flux Branching

In [ ]:
import pandas as pd
import numpy as np
import re
import os
from datetime import date

# loading matplotlib, a library for visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
from cobra.io import read_sbml_model, write_sbml_model, save_matlab_model, save_json_model
from cobra import Reaction

from iambcodes.cobra import *

Today = date.today().strftime('%y%m%d')
FigureType = 'svg'

## Defining support functions for analysis of GSMM iUma22v1.1

In [ ]:
def CoSub(Flux, Cnum, R):
    '''
    Defining the ratios of cosubstrates.
    
    Arguments:
    Flux: float, total C-mole flux
    C1: int, carbon atoms in substrate 1
    C2: int, carbon atoms in substrate 2
    R: float, flux ratio of both substrate: R = C1*F1 / (C2*F2)
    
    Return:
    F1, F2: float, molar concentrations of substrate 1 and substrate 2.
    '''
    F1 = Flux / (Cnum[0]*(1+1/R))
    F2 = F1*Cnum[0] / (R*Cnum[1])
    return F1, F2

def countCinFormula(formula):
    '''
    Counting C-atoms of metabolite in cobra model
    
    Argument:
    formula: string, the metabolite formula from the cobra model
    
    Return:
    int, number of C-atoms in formula
    '''
    return int(re.search(r'C(.*)H',formula).group(1)) if re.search(r'C(.*)H',formula).group(1) else int(1)

def set_ObjiUma(model, Product):
    '''
    Adding boundary flux for a desired product and setting the objective reaction to the product in the model.
    
    Arguments:
    model: cobra model, iUma22
    Product: string, model metabolite ID
    
    Return:
    cobra model, updated with a demand flow for the product and optimization of the new demand reaction.
    '''
    model.add_boundary(model.metabolites.get_by_id(Product), type='demand')
    model.objective = 'DM_{}'.format(Product)
    return model

def set_SubiUma(model, Sub, Flux, Ratio):
    '''
    Setting the desired substrate in the model.
    
    Arguments:
    model: cobra model, iUma22
    Sub: string/list, model metabolite ID
    Flux: float, desired substrate uptake rate
    Ratio: float, when Sub is a list, then uptake rates are balanced according to the Ratio.
    
    Return:
    cobra model, updated with designed (co-) substrate uptake rates.
    '''
    Catoms = [countCinFormula(model.metabolites.get_by_id(subi).formula) for subi in Sub]
    Cup = Flux/np.sum(Catoms)
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
    if len(Sub)<2:
        model.reactions.get_by_id('EX_{}'.format(Sub[0])).lower_bound = -Cup
    else:
        C1 = Catoms[0]
        C2 = Catoms[1]
        F1,F2 = CoSub(Flux, C1, C2, Ratio)
        for subi, fluxi in zip(Sub, [F1,F2]):
            model.reactions.get_by_id('EX_{}'.format(subi)).lower_bound = -fluxi
    return model

def Sim_iUma(model, CFlux, Substrates, Ratio, Objective=None):
    '''
    Simulating a model with multiple substrates

    Arguments:
    model: cobra model, iUma22
    CFlux: float, C-flux total
    Substrates: string/list, model metabolite ID
    Ratio: float, when Sub is a list, then uptake rates are balanced according to the Ratio.
    Objective (optional): string, reaction to be optimized
    
    Return:
    Flux table from model.optimize
    '''
    Catoms = [countCinFormula(model.metabolites.get_by_id(subi).formula) for subi in Substrates]
    SubFlux = CoSub(CFlux,Catoms,Ratio)
    # assuming the substrate exists as extracellular compound with exchange reaction following the syntax
    SubEX = ['EX_{}'.format(subi) for subi in Substrates]
# #     print(count)
#     count += 1
#     print('glc init:{}, ace init:{}'.format(glc_up, ace_up))
    with model as model:
        model.reactions.get_by_id(SubEX[0]).lower_bound = -SubFlux[0]
        model.reactions.get_by_id(SubEX[1]).lower_bound = -SubFlux[1]
#         model = set_ObjiUma(model, 'succ_c')
        if Objective:
            model.objective = Objective # 'EX_itacon_e'
#         print(model.summary())
        Flux = model.optimize().fluxes
#     print('glc actual:{}, ace actual:{}'.format(mySol[-1]['EX_glc__D_e'], mySol[-1]['EX_ac_e']))
#     print('Simulated total C-Flux: ',6*mySol[-1]['EX_glc__D_e'] + 2*mySol[-1]['EX_ac_e'])
# #     print(glc_up, ace_up)
    return Flux


## Loading iUma22v1.1 model

In [ ]:
# generating cobra variable from SBML/xml file
# assuming the base path is in 'model' of a cloned git-directory of iUma22.
ModelDir = os.path.join('..','model')
ModelFile = 'iUma22v1.1.xml'
iUmaFile = os.path.join(ModelDir,ModelFile)
model = read_sbml_model(iUmaFile)

## Simulations of co-substrate uptake

In [ ]:
CFlux = 13.2
Substrates = ['glc__D_e', 'ac_e']
Objective = 'EX_itacon_e'
count = 1
Samples = 50
ratio = np.logspace(1,-1,Samples)
mySol = list()

for iratio in ratio:
    Flux = Sim_iUma(model, CFlux, Substrates, iratio, Objective)
    mySol.append(Flux)



## Reformatting data

In [ ]:
Res = list()

for indx in range(len(mySol)):
    Ctot = np.abs(6*mySol[indx]['EX_glc__D_e'] + 2*mySol[indx]['EX_ac_e'])
    glc = np.abs(6*mySol[indx]['EX_glc__D_e'])
    ace = np.abs(2*mySol[indx]['EX_ac_e'])
    AcTot = ace/Ctot
    ita = np.abs(5*mySol[indx]['EX_itacon_e'])
    iy = ita/Ctot
    iace = ita*AcTot
    iaceY = iy*AcTot
    bio = np.abs(mySol[indx]['BIOMASS_REACTION'])
    Resi = {'C-Flux': Ctot, 'Glc':glc, 'Ace':ace, 'AceRel': AcTot, 'Ita':ita, 'Ita C-Yield':iy, 'ItaYield_x_AceRel':iaceY, 'Ita_x_AceRel':iace, 'Growth':bio} 
    Res.append(Resi.copy())
#     Res[indx][1] = np.abs(6*mySol[indx]['EX_glc__D_e'])
#     Res[indx][2] = np.abs(2*mySol[indx]['EX_ac_e'])
res_df = pd.DataFrame(Res)

## Visualization of flux results

### General settings

In [ ]:
RatioNumb = [22,32,43]
DataDiv = [np.arange(0,RatioNumb[0]),np.arange(RatioNumb[0],RatioNumb[1]),np.arange(RatioNumb[1],RatioNumb[2]),np.arange(RatioNumb[2], Samples)]
colors = ['r', 'b', 'g','c','m'] # , 'k', 'g', 'm'
msize = 100
matplotlib.rcParams.update({'font.size': 15})
# ticks inside:
# https://stackoverflow.com/questions/47873389/matplotlib-move-tick-labels-inside-plot-area

### Plot Total Carbon VS Acetate Carbon Ratio

In [ ]:
XVal = 'AceRel'
FigureName = 'AceRel-VS-CTotalGlcAce'
mkr_dict = {'Glc': 'x', 'Ace': '+'} #'replace': 'x', 'drop': 'o'
for Target in mkr_dict.keys():
    for midx, Range in enumerate(DataDiv):
        plt.scatter(res_df.iloc[Range][XVal]*100, res_df.iloc[Range][Target], 
                    s = msize, 
                    c = colors[midx], 
                    marker = mkr_dict[Target],
                    label = Target)
plt.xlabel('Acetate Carbon/ total Carbon, %')
plt.ylabel('Carbon flux, mmol(C)/gDW/h)')
plt.legend()
plt.tick_params(direction="in")
plt.savefig('{}_{}.{}'.format(Today,FigureName,FigureType))
plt.show()

In [ ]:
XVal = 'AceRel'
FigureName = 'AceRel-VS-ItaYield'
mkr_dict = {'ItaYield_x_AceRel': 'x', 'Ita C-Yield': '+'} #'replace': 'x', 'drop': 'o'
for Target in mkr_dict.keys():
    for midx, Range in enumerate(DataDiv):
        plt.scatter(res_df.iloc[Range][XVal]*100, res_df.iloc[Range][Target], 
                    s = msize, 
                    c = colors[midx], 
                    marker = mkr_dict[Target])
plt.xlabel('Acetate Carbon/ total Carbon, %')
plt.ylabel('Itaconate yield, mol/mol')
plt.tick_params(direction="in")
plt.savefig('{}_{}.{}'.format(Today,FigureName,FigureType))
plt.show()

In [ ]:
RctFluxName = ['HEX1', 
               'G6PDH', 
               'MDH', 
               'TRANS-RXNTFS-7-ACET1018', 
               'CSNATr', 
               'ACONTa', 
               'ISOCIT-CLEAV-RXN', 
               'SUCDH_q6_m', 
               'MDHm',
               'MALSm', 
               'TRANS_Malate_Phosphate_Shuttle', 
               'TRANS_Cis_Aconitate_Malate_Shuttle', 
               'Putative_Trans_Aconitate_Decarboxylase', 
               'EX_co2_e',
               'EX_itacon_e']
CDict = {'HEX1':6, 
         'G6PDH':6, 
         'MDH':4, 
         'TRANS-RXNTFS-7-ACET1018':2, 
         'CSNATr':2, 
         'ACONTa':6, 
         'ISOCIT-CLEAV-RXN':6, 
         'MALSm':4, 
         'SUCDH_q6_m':4, 
         'MDHm':4,
         'TRANS_Malate_Phosphate_Shuttle':4, 
         'TRANS_Cis_Aconitate_Malate_Shuttle':4, 
         'Putative_Trans_Aconitate_Decarboxylase':1, 
         'EX_co2_e':1,
         'EX_itacon_e':5}

In [ ]:
for ratio in RatioNumb:
    Ctot = 6*mySol[ratio]['EX_glc__D_e']+2*mySol[ratio]['EX_ac_e']
    exp4all = mySol[ratio]/np.abs(Ctot)*100
    print('#################################################################################')
    print(f'Normalized 100 C-flux for ratio index {ratio}')
    [print(f'{model.reactions.get_by_id(Rct).id} \t {model.reactions.get_by_id(Rct).reaction} \t {round(exp4all[Rct]*CDict[Rct])}') for Rct in RctFluxName];

In [ ]:
exp = 22 # 9 10 11
metid = 'co2_e' 
# itacon_c, glc__D_c, g6p_c, actp_c, cit_m, fum_m, succ_m, 2mcit_c, 2obut_c,thr__L_c,hom__L_c, asp__L_c,oaa_c, accoa_c, malcoa_c
Ctot = 6*mySol[exp]['EX_glc__D_e']+2*mySol[exp]['EX_ac_e']
exp4all = mySol[exp]/np.abs(Ctot)*100
# exp4all[flux]
print('total C-Flux: ', round(np.abs(Ctot)))
[print(f'{rct.id}\t{rct.reaction}\t{round(exp4all[rct.id])}') for rct in model.metabolites.get_by_id(metid).reactions if np.abs(exp4all[rct.id])>.1];

In [ ]:
XmlNew = os.path.join(BaseDir, 'iUma22v1.1.xml')
MatNew = os.path.join(BaseDir, 'iUma22v1.1.mat')
JsonNew = os.path.join(BaseDir, 'iUma22v1.1.json')
write_sbml_model(model, XmlNew)
save_matlab_model(model, MatNew)
save_json_model(model, JsonNew)
print('New model sbml stored as: {}'.format(XmlNew))


In [ ]:
KO = ['MDH', 'HISTD', 'TRANS_Adenine_Nucleotide_Transporter']
[print(model.reactions.get_by_id(myKO).reaction) for myKO in KO]
GlcFlux = 56/6
AceFlux = 44/2
with model as model:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = -GlcFlux
    model.reactions.get_by_id('EX_ac_e').lower_bound = -AceFlux
    [setattr(model.reactions.get_by_id(myKO), 'bounds',[0,0]) for myKO in KO]
    model.objective = 'EX_itacon_e'
    KOSol = model.optimize()
Ctot = 6*KOSol.fluxes['EX_glc__D_e']+2*KOSol.fluxes['EX_ac_e']
exp4all = KOSol.fluxes/np.abs(Ctot)*100
print('#################################################################################')
print(f'Normalized 100 C-flux for ratio index {ratio}')
[print(f'{model.reactions.get_by_id(Rct).id} \t {model.reactions.get_by_id(Rct).reaction} \t {round(exp4all[Rct]*CDict[Rct])}') for Rct in RctFluxName];

In [ ]:
metid = 'ac_c'
[print('{}\t{}'.format(rct.id, rct.reaction)) for rct in model.metabolites.get_by_id(metid).reactions];

In [ ]:
model.reactions.get_by_id('PPCKm')

In [ ]:
# [reaction for reaction in model.metabolites.get_by_id('mal__L_c').reactions if reaction in model.metabolites.get_by_id('pyr_c').reactions]
Met1 = 'icit_c'
Met2 = 'glx_c' # 'CPD-356_c'
# print(MetSharedRcts(model,Met1, Met2)[0]) # 'shared reactions of {} and {}: {}'.format(Met1, Met2, MetSharedRcts(model,'man_c', 'MANNOSE-6P_c')))
flux = MetFluxConnect(model, Met1, Met2, slim=False)
print('possible flux between {} and {}: {}'.format(Met1, Met2, MetFluxConnect(model, Met1, Met2)))

In [ ]:
model.metabolites.get_by_id('histd_c')
# model.summary()

In [ ]:
with model as model:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = -2
    model.reactions.get_by_id('EX_ac_e').lower_bound = -4
    # model = set_ObjiUma(model, 'succ_c')
#     model.objective = 'EX_itacon_e'
    print(model.summary())
    print(model.metabolites.get_by_id('accoa_c').summary())

In [ ]:
model.reactions.get_by_id('MALSc')